<a href="https://colab.research.google.com/github/hucarlos08/GEO-ML/blob/main/CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GridSearchCV
GridSearchCV is a technique for tuning hyperparameters, seeking to find the optimal hyperparameters for a model in a systematic way. The idea is to define a grid of hyperparameters, where each point in the grid is a specific configuration of hyperparameters. The model is then trained and evaluated for each of these configurations, and the configuration that yields the best performance according to some metric (e.g., accuracy for classification problems, mean squared error for regression problems) is chosen as the optimal one.

Let's break it down:

- **Grid**: This refers to the combination of different hyperparameters that you'd like to test. For example, if you're tuning a support vector machine, you might want to test different values for the cost parameter C and the kernel gamma. You could define C to take on values from the set {1, 10, 100} and gamma from the set {0.1, 0.01}. Your grid would then consist of 9 combinations: (C=1, gamma=0.1), (C=1, gamma=0.01), (C=10, gamma=0.1), etc.

- **Search**: This is the process of training and evaluating a model for each point in the grid. Typically, this is done using some form of cross-validation to get a reliable estimate of the model's performance.

- **CV**: Stands for Cross-Validation. In cross-validation, the training set is split into k smaller sets or "folds". The model is then trained on k-1 of these folds and evaluated on the remaining fold. This process is repeated k times, with a different fold used for evaluation each time. The performance of the model is then the average performance over these k folds. This helps to get a more robust estimate of the model's performance and to mitigate overfitting.

By systematically exploring the grid of hyperparameters with cross-validation, **GridSearchCV** can find a good combination of hyperparameters, balancing model complexity and model performance in a principled way. The main drawback of this method is that it can be computationally expensive, especially if the grid is large and the model takes a long time to train.

In [28]:
# 2. Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

# 3. Load and explore the data
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# A simple pipeline, with only two steps
pipeline = Pipeline([
    ('normalizer', StandardScaler()), #Step1 - normalize data
    ('clf', LogisticRegression(max_iter=10000)) #Step2 - classifier
])


# 7. Train and evaluate the model using the Pipeline
# Train the model
pipeline.fit(X_train, y_train)

# Predict the classes
y_pred = pipeline.predict(X_test)

# Evaluate the model performance
print('Accuracy: {:.2f}'.format(metrics.accuracy_score(y_test, y_pred)))

# 8. Hyperparameter tuning in a Pipeline
# Create a parameter grid
param_grid = {
    'clf__penalty': ['l1', 'l2'],
    'clf__C': np.logspace(-4, 4, 20),
    'clf__solver': ['liblinear', 'saga']
}

# Create a GridSearch object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=0)

# Fit the GridSearch object
best_model = grid_search.fit(X_train, y_train)

# Get the results into a dataframe
results = pd.DataFrame(best_model.cv_results_)

# Print the best parameters
print('Best Parameters: ', best_model.best_params_)

# Predict the classes using the best model
y_pred_best = best_model.predict(X_test)

# Evaluate the best model performance
print('Accuracy of Best Model: {:.2f}'.format(metrics.accuracy_score(y_test, y_pred_best)))


Accuracy: 1.00
Best Parameters:  {'clf__C': 0.615848211066026, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
Accuracy of Best Model: 1.00


In [29]:
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__C,param_clf__penalty,param_clf__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004629,0.002962,0.001100,0.000140,0.0001,l1,liblinear,"{'clf__C': 0.0001, 'clf__penalty': 'l1', 'clf_...",0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.000000,71
1,0.004492,0.001837,0.001115,0.000111,0.0001,l1,saga,"{'clf__C': 0.0001, 'clf__penalty': 'l1', 'clf_...",0.375000,0.333333,0.333333,0.333333,0.333333,0.341667,0.016667,69
2,0.010635,0.004860,0.004456,0.004395,0.0001,l2,liblinear,"{'clf__C': 0.0001, 'clf__penalty': 'l2', 'clf_...",0.708333,0.916667,0.791667,0.791667,0.833333,0.808333,0.067700,59
3,0.005634,0.002049,0.002673,0.003068,0.0001,l2,saga,"{'clf__C': 0.0001, 'clf__penalty': 'l2', 'clf_...",0.625000,0.333333,0.333333,0.333333,0.333333,0.391667,0.116667,68
4,0.006519,0.005563,0.001198,0.000225,0.000264,l1,liblinear,"{'clf__C': 0.00026366508987303583, 'clf__penal...",0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.000000,71


In [26]:
import plotly.express as px

# Select the columns we are interested in
subset_results = results[['param_clf__C', 'param_clf__penalty', 'param_clf__solver', 'mean_test_score']]

# Rename columns for a more meaningful plot
subset_results = subset_results.rename(columns={
    'param_clf__C': 'C',
    'param_clf__penalty': 'Penalty',
    'param_clf__solver': 'Solver',
    'mean_test_score': 'Mean Test Score'
})

# Convert columns to appropriate dtypes for plotting
subset_results['C'] = subset_results['C'].astype(float)
subset_results['Mean Test Score'] = subset_results['Mean Test Score'].astype(float)
subset_results['Penalty'] = subset_results['Penalty'].map({'l1': 1, 'l2': 2})
subset_results['Solver'] = subset_results['Solver'].map({'liblinear': 1, 'saga': 2})


# Make the parallel coordinates plot
fig = px.parallel_coordinates(
    subset_results,
    color="Mean Test Score",
    labels={"C": "C", "Penalty": "Penalty", "Solver": "Solver", "Mean Test Score": "Mean Test Score"},
    color_continuous_scale=px.colors.diverging.Tealrose,
    color_continuous_midpoint=subset_results['Mean Test Score'].mean()
)
fig.show()


In [21]:
subset_results.head(10)

,C,Penalty,Solver,Mean Test Score
0,0.000100,1,1,0.333333
1,0.000100,1,2,0.341667
2,0.000100,2,1,0.808333
3,0.000100,2,2,0.408333
4,0.000264,1,1,0.333333
5,0.000264,1,2,0.325000
6,0.000264,2,1,0.808333
7,0.000264,2,2,0.666667
8,0.000695,1,1,0.333333
9,0.000695,1,2,0.325000
